In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame(columns=['create', 'deliver', 'destory', 'material'])
df

,create,deliver,destory,material


In [4]:
class IssueRecorder():
    def __init__(self):
        # ID prefix
        self.prefix = 'Z-L19'
        # Main dataframe
        self.df = pd.DataFrame(columns=['create', 'deliver', 'destory', 'material'])
    
    def append(self, date, idxs, opt, opt_date, material='--'):
        # Append new issue
        # date: date of ID
        # idxs: list of ID index
        # opt: operation: create, deliver, destory
        # opt_date: operating date
        
        # idxs should be a list
        if type(idxs) is int:
            idxs = [idxs]
            
        # For each index
        for idx in idxs:
            # Issue name
            name = '{prefix}-{date}-{idx:03d}'.format(prefix=self.prefix, date=date, idx=idx)
            print(name)
            # Build series
            se = pd.Series(name=name, data={opt: opt_date, 'material': material})
            print(se)
            # Append series into main dataframe
            self.df = self.df.append(se)
            
    def check(self):
        # Walk throught main dataframe, and check issues
        self.df = self.df.fillna('--')
        # Init bug list
        self.bugs = []
        self.checked = pd.DataFrame(columns=['create', 'deliver', 'destory'])
        # For each issue name
        for name in self.df.index.unique():
            # There should be several records for certain issue name
            ses = ir.df.loc[name]
            print(ses)
            # If closed well, print pass,
            # if not, print bug and record into bug list
            if self.check_finish(ses):  # len(ses) == 2:
                print('Pass.')
            else:
                print('Bug.')
                self.bugs.append(name)
    
    def check_finish(self, ses):
        # Check if the session closed
        # ses: a series of certain issue name
        
        # name: Issue name
        name = ses.index.unique()
        if len(name) > 1:
            return False
        name = name[0]
        
        # A closed session should contain exactly 2 records
        if not len(ses) == 2:
            return False
        
        # a, b: two records
        a = ses.iloc[0].to_dict()
        b = ses.iloc[1].to_dict()
        
        # The issue in a closed session should not be created twice
        if all([a['create'] == '--', b['create'] == '--']):
            return False
        
        # data: data of circle report of the issue
        data = dict(
            create = '--',
            deliver = '--',
            destory = '--',
        )
        # Fill data using a and b
        if a['create'] == '--':
            data['create'] = b['create']
            if a['destory'] == '--':
                data['deliver'] = a['deliver']
            else:
                data['destory'] = a['destory']
        else:
            data['create'] = a['create']
            if b['destory'] == '--':
                data['deliver'] = b['deliver']
            else:
                data['destory'] = b['destory']
        
        # Check if creating date is in front of delivering or destorying.
        d = 'deliver'
        if data['deliver'] == '--':
            d = 'destory'
            if data['create'] > data[d]:
                return False
            
        # The issue is fine, record it into checked
        self.checked = self.checked.append(pd.Series(name=name, data=data))
        return True
    
    def logging(self, message, fp, to_html=False):
        print(message)
        if to_html:
            message = message.to_html()
        print('\n'.join(['<div>', message, '</div>']), file=fp)
    
    def report_bugs(self):
        with open('bugs.html', 'w') as fp:
            self.logging('=' * 80, fp)
            self.logging('Bugs report.', fp)
            for name in self.bugs:
                self.logging('-' * 80, fp)
                self.logging(self.df.loc[name], fp, to_html=True)

In [5]:
ir = IssueRecorder()
ir.df

,create,deliver,destory,material


In [6]:
####################################################
opt = 'create'

date = '20181229'
[ir.append(date, idxs, opt, date, driver)
 for idxs, driver in [[range(1, 7), 'paper'],
                      [range(7, 9), 'paper'],
                      [range(9, 11), 'paper'],
                      [range(11, 13), 'cd'],
                      [13, 'paper']]]

date = '20190102'
[ir.append(date, idxs, opt, date, driver)
 for idxs, driver in [[range(1, 7), 'paper'],
                      [range(7, 9), 'paper'],
                      [9, 'cd']]]

date = '20190103'
[ir.append(date, idxs, opt, date, driver)
 for idxs, driver in [[range(1, 3), 'paper'],
                      [3, 'cd']]]

date = '20190319'
[ir.append(date, idxs, opt, date, driver)
 for idxs, driver in [[range(1, 5), 'paper'],
                      [5, 'cd'],
                      [6, 'paper']]]

####################################################
opt = 'deliver'

opt_date = '20190612'
[ir.append('20181229', idxs, opt, opt_date)
 for idxs in [range(1, 7),
              range(7, 9),
              range(9, 11),
              12]]

opt_date = '20190320'
[ir.append('20190319', idxs, opt, opt_date)
 for idxs in [range(1, 5),
              5,
              6]]

####################################################
opt = 'destory'

opt_date = '20190424'
[ir.append('20190319', idxs, opt, opt_date)
 for idxs in [5]]

opt_date = '20190130'
[ir.append('20181229', idxs, opt, opt_date)
 for idxs in [range(11, 13),
              13,
              range(9, 11)]]
[ir.append('20190102', idxs, opt, opt_date)
 for idxs in [9,
              range(1, 7),
              range(7, 9)]]

#####################################################
opt = 'create'

opt_date = '20191025'
[ir.append('20191025', idxs, opt, opt_date)
for idxs in range(47, 51)]
[ir.append('20191025', idxs, opt, opt_date)
for idxs in range(51, 57)]

#####################################################
opt = 'destory'

opt_data = '20190125'
[ir.append('20191025', idxs, opt, opt_date)
for idxs in [47, 48]]

#####################################################
opt = 'deliver'

opt_date = '20191026'
[ir.append('20191025', idxs, opt, opt_date)
for idxs in [49, 50]]
[ir.append('20191025', idxs, opt, opt_date)
for idxs in range(51, 57)]

#####################################################
opt = 'create'

opt_date = '20191104'
[ir.append('20191104', idxs, opt, opt_date)
for idxs in [1, 2]]
[ir.append('20191104', idxs, opt, opt_date)
for idxs in range(3, 8)]

#####################################################
opt = 'deliver'

opt_date = '20191104'
[ir.append('20191104', idxs, opt, opt_date)
for idxs in [1, 2]]
[ir.append('20191104', idxs, opt, opt_date)
for idxs in range(3, 8)]

#####################################################
opt = 'deliver'

opt_date = '20190612'
[ir.append('20190103', idxs, opt, opt_date)
for idxs in range(1, 4)]

#####################################################
opt = 'create'

opt_date = '20191129'
[ir.append('20191129', idxs, opt, opt_date)
for idxs in range(2, 6)]
[ir.append('20191129', idxs, opt, opt_date)
for idxs in range(6, 18)]

#####################################################
opt = 'create'

opt_date = '20191129'
[ir.append('20191129', idxs, opt, opt_date)
for idxs in range(18, 22)]
[ir.append('20191129', idxs, opt, opt_date)
for idxs in range(22, 33)]

#####################################################
opt = 'destory'

opt_date = '20191129'
[ir.append('20191129', idxs, opt, opt_date)
for idxs in range(18, 22)]
[ir.append('20191129', idxs, opt, opt_date)
for idxs in range(22, 33)]

#####################################################
opt = 'deliver'

opt_date = '20191129'
[ir.append('20191129', idxs, opt, opt_date)
for idxs in range(2, 6)]
[ir.append('20191129', idxs, opt, opt_date)
for idxs in range(6, 18)]

#####################################################
opt = 'create'

opt_date = '20191206'
[ir.append('20191206', idxs, opt, opt_date)
for idxs in [2, 3, 4]]

#####################################################
opt = 'deliver'

opt_date = '20191209'
[ir.append('20191206', idxs, opt, opt_date)
for idxs in [2, 3, 4]]

######################################################
opt = 'create'

opt_date = '20191223'
[ir.append('20191223', idxs, opt, opt_date)
for idxs in [1]]

######################################################
opt = 'deliver'

opt_date = '20191223'
[ir.append('20191223', idxs, opt, opt_date)
for idxs in [1]]

print('done')

Z-L19-20181229-001
create      20181229
material       paper
Name: Z-L19-20181229-001, dtype: object
Z-L19-20181229-002
create      20181229
material       paper
Name: Z-L19-20181229-002, dtype: object
Z-L19-20181229-003
create      20181229
material       paper
Name: Z-L19-20181229-003, dtype: object
Z-L19-20181229-004
create      20181229
material       paper
Name: Z-L19-20181229-004, dtype: object
Z-L19-20181229-005
create      20181229
material       paper
Name: Z-L19-20181229-005, dtype: object
Z-L19-20181229-006
create      20181229
material       paper
Name: Z-L19-20181229-006, dtype: object
Z-L19-20181229-007
create      20181229
material       paper
Name: Z-L19-20181229-007, dtype: object
Z-L19-20181229-008
create      20181229
material       paper
Name: Z-L19-20181229-008, dtype: object
Z-L19-20181229-009
create      20181229
material       paper
Name: Z-L19-20181229-009, dtype: object
Z-L19-20181229-010
create      20181229
material       paper
Name: Z-L19-20181229-010, dtyp

In [7]:
ir.check()

create   deliver destory material
Z-L19-20181229-001  20181229        --      --    paper
Z-L19-20181229-001        --  20190612      --       --
Pass.
                      create   deliver destory material
Z-L19-20181229-002  20181229        --      --    paper
Z-L19-20181229-002        --  20190612      --       --
Pass.
                      create   deliver destory material
Z-L19-20181229-003  20181229        --      --    paper
Z-L19-20181229-003        --  20190612      --       --
Pass.
                      create   deliver destory material
Z-L19-20181229-004  20181229        --      --    paper
Z-L19-20181229-004        --  20190612      --       --
Pass.
                      create   deliver destory material
Z-L19-20181229-005  20181229        --      --    paper
Z-L19-20181229-005        --  20190612      --       --
Pass.
                      create   deliver destory material
Z-L19-20181229-006  20181229        --      --    paper
Z-L19-20181229-006        --  20190612  

In [8]:
ir.checked

,create,deliver,destory
Z-L19-20181229-001,20181229,20190612,--
Z-L19-20181229-002,20181229,20190612,--
Z-L19-20181229-003,20181229,20190612,--
Z-L19-20181229-004,20181229,20190612,--
Z-L19-20181229-005,20181229,20190612,--
...,...,...,...
Z-L19-20191129-032,20191129,--,20191129
Z-L19-20191206-002,20191206,20191209,--
Z-L19-20191206-003,20191206,20191209,--
Z-L19-20191206-004,20191206,20191209,--


In [9]:
ir.report_bugs()

Bugs report.
--------------------------------------------------------------------------------
                      create   deliver   destory material
Z-L19-20181229-009  20181229        --        --    paper
Z-L19-20181229-009        --  20190612        --       --
Z-L19-20181229-009        --        --  20190130       --
--------------------------------------------------------------------------------
                      create   deliver   destory material
Z-L19-20181229-010  20181229        --        --    paper
Z-L19-20181229-010        --  20190612        --       --
Z-L19-20181229-010        --        --  20190130       --
--------------------------------------------------------------------------------
                      create   deliver   destory material
Z-L19-20181229-012  20181229        --        --       cd
Z-L19-20181229-012        --  20190612        --       --
Z-L19-20181229-012        --        --  20190130       --
------------------------------------------------

In [11]:
with open('checked.html', 'w') as fp:
    ir.checked.to_html(fp, index=True)